- https://duckdb.org/docs/api/python/overview 
- https://duckdb.org/docs/guides/sql_editors/dbeaver 
- https://duckdb.org/docs/guides/sql_editors/dbeaver.html

In [3]:
import duckdb

In [4]:
r1 = duckdb.sql('SELECT 42 AS i')
r1

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘

In [5]:
duckdb.sql("SELECT i*100 AS k FROM r1")

┌───────┐
│   k   │
│ int32 │
├───────┤
│  4200 │
└───────┘

In [6]:
duckdb.sql("SELECT i*100 FROM r1")

┌───────────┐
│ (i * 100) │
│   int32   │
├───────────┤
│      4200 │
└───────────┘

In [9]:
duckdb.read_csv("sample.csv")

┌─────────┬─────────────┬─────────────┬─────────────┐
│ column0 │      a      │      b      │      c      │
│  int64  │   double    │   double    │   double    │
├─────────┼─────────────┼─────────────┼─────────────┤
│       1 │ 0.051017541 │ 0.829733518 │ 0.191106993 │
│       2 │ 0.034442112 │ 0.313852899 │ 0.593982669 │
│       3 │ 0.980742868 │ 0.517989292 │ 0.706687286 │
└─────────┴─────────────┴─────────────┴─────────────┘

In [16]:
import pandas as pd
df = pd.read_csv('sample.csv').iloc[:, 1:]
df

,a,b,c
0,0.051018,0.829734,0.191107
1,0.034442,0.313853,0.593983
2,0.980743,0.517989,0.706687


In [40]:
con = duckdb.connect(database='file.db')
con.sql('CREATE TABLE test (i INTEGER)')
con.sql('INSERT INTO test VALUES (42)')


In [41]:
con.table('test').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [42]:
con.sql('INSERT INTO test VALUES (44)')

In [43]:
con.table('test').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    44 │
└───────┘



In [51]:
with duckdb.connect('file.db') as con:
    #con.sql('CREATE TABLE test (i INTEGER)')
    con.sql('INSERT INTO test VALUES (42)')
    con.table('test').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
│    44 │
│    42 │
└───────┘



In [52]:
with duckdb.connect('file.db') as con:
    for i in range(50, 100):
        con.sql(f'INSERT INTO test VALUES ({i})')
    con.table('test').show()

┌────────────┐
│     i      │
│   int32    │
├────────────┤
│         42 │
│         44 │
│         42 │
│         50 │
│         51 │
│         52 │
│         53 │
│         54 │
│         55 │
│         56 │
│          · │
│          · │
│          · │
│         90 │
│         91 │
│         92 │
│         93 │
│         94 │
│         95 │
│         96 │
│         97 │
│         98 │
│         99 │
├────────────┤
│  53 rows   │
│ (20 shown) │
└────────────┘



In [53]:
con.table('test').show()

ConnectionException: Connection Error: Connection has already been closed

In [54]:
# re connect
con = duckdb.connect(database='file.db')
con.table('test').show()

┌────────────┐
│     i      │
│   int32    │
├────────────┤
│         42 │
│         44 │
│         42 │
│         50 │
│         51 │
│         52 │
│         53 │
│         54 │
│         55 │
│         56 │
│          · │
│          · │
│          · │
│         90 │
│         91 │
│         92 │
│         93 │
│         94 │
│         95 │
│         96 │
│         97 │
│         98 │
│         99 │
├────────────┤
│  53 rows   │
│ (20 shown) │
└────────────┘



- **The test table will be listed under the main schema in the DBeaver interface, which is the default schema where tables are usually found in DuckDB.**

In [59]:
# Dbeaver에서 조회하려면 close()하고 확인
# Closing the connection
con.close()

In [61]:
con = duckdb.connect(database='file.db')
result = con.execute('SELECT * FROM test').fetchdf()
print(result)

     i
0   42
1   44
2   42
3   50
4   51
5   52
6   53
7   54
8   55
9   56
10  57
11  58
12  59
13  60
14  61
15  62
16  63
17  64
18  65
19  66
20  67
21  68
22  69
23  70
24  71
25  72
26  73
27  74
28  75
29  76
30  77
31  78
32  79
33  80
34  81
35  82
36  83
37  84
38  85
39  86
40  87
41  88
42  89
43  90
44  91
45  92
46  93
47  94
48  95
49  96
50  97
51  98
52  99


In [62]:
# Connect to DuckDB (in-memory database)
con = duckdb.connect(database=':memory:')

con = duckdb.connect(database='file.db')

# Write the DataFrame to a DuckDB table
df.to_sql('test', con, if_exists='replace', index=False)

# Query the table to confirm the write operation
result = con.execute('SELECT * FROM test').fetchdf()
print(result)

# Close the connection
con.close()

          a         b         c
0  0.051018  0.829733  0.191107
1  0.034442  0.313853  0.593983
2  0.980743  0.517989  0.706687


C:\Users\조근하\AppData\Local\Temp\ipykernel_31036\866922225.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('test', con, if_exists='replace', index=False)


In [63]:
with duckdb.connect(database='scratch01.db') as con:
    # Write the DataFrame to a DuckDB table
    df.to_sql('sf', con, if_exists='replace', index=False)
    # Query the table to confirm the write operation
    result = con.execute('SELECT * FROM sf').fetchdf()
    
print(result)


          a         b         c
0  0.051018  0.829733  0.191107
1  0.034442  0.313853  0.593983
2  0.980743  0.517989  0.706687


C:\Users\조근하\AppData\Local\Temp\ipykernel_31036\2469124658.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('sf', con, if_exists='replace', index=False)


In [64]:
con = duckdb.connect(database='file.db')
result = con.execute('SELECT * FROM test').fetchdf()
print(result)

          a         b         c
0  0.051018  0.829733  0.191107
1  0.034442  0.313853  0.593983
2  0.980743  0.517989  0.706687


In [65]:
con.close()